# AI-Powered Customer Support Automation Email Agent

## Problem Statement

In e-commerce, customers often send emails or chat queries regarding refunds, EMI options, shipping delays, account issues, or technical problems. 

Manual customer support is:
- Slow and inconsistent
- Error-prone
- Lacks personalization

Our goal is to **automate email responses** with an intelligent AI agent that can:
- Understand the intent behind queries
- Detect emotional tone and urgency
- Retrieve knowledge from internal knowledge bases and online sources
- Draft high-quality, context-aware responses
- Maintain multi-turn session memory for personalized interactions


## Solution & Innovation

We have built a **multi-agent AI system** to solve these challenges:

- **Multi-Agent System:**
  - `ResponseAgent` (drafts replies using Gemini LLM)
  - `QAAgent` (reviews responses for quality)
  - `ToneCheckerTool` (detects emotional tone and urgency)
- **Knowledge Retrieval:**
  - Internal KB using embeddings + TF-IDF
  - Fallback to Google Search for missing information
- **Customer Memory & Session:**
  - Stores conversation history for multi-turn queries
- **Custom Tools:**
  - Product Info Tool for EMI, warranty, and product features

**Innovation Highlights:**
- Personalized automated responses
- Real-time knowledge retrieval
- Intelligent multi-agent orchestration
- EMI calculation and explanation per product


# Importing The Necessary libraries 

In [19]:
# imports and secret loading (Kaggle-friendly)
import os
import json
import time
from pathlib import Path
from typing import List, Dict, Optional, Tuple

import numpy as np
import pandas as pd

# ML libs
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity

# Load API key (Kaggle secrets preferred)
GENAI_API_KEY = None
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    GENAI_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    print("Loaded GENAI_API_KEY from Kaggle Secrets:", GENAI_API_KEY is not None)
except Exception:
    GENAI_API_KEY = os.environ.get("GENAI_API_KEY") or os.environ.get("GOOGLE_API_KEY")
    print("Loaded GENAI_API_KEY from env:", GENAI_API_KEY is not None)

# Try import Gemini SDK
GENAI_SDK = False
genai = None
try:
    import google.generativeai as genai
    GENAI_SDK = True
    print("google.generativeai SDK available.")
except Exception as e:
    GENAI_SDK = False
    genai = None
    print("google.generativeai not available:", e)

# Final availability flag
GENAI_AVAILABLE = GENAI_SDK and (GENAI_API_KEY is not None)
print("GENAI_AVAILABLE =", GENAI_AVAILABLE)


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Loaded GENAI_API_KEY from Kaggle Secrets: True
google.generativeai SDK available.
GENAI_AVAILABLE = True


# Initializing the GOOGLE_API_KEY

In [20]:
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai

# Load key
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GOOGLE_API_KEY")

# Apply it
genai.configure(api_key=api_key)

print("Gemini API Ready")



Gemini API Ready


In [21]:
# Cell 3: Knowledge Base (use 'id', 'title', 'content')
KB = [
    {"id": "kb1", "title": "Refund policy", "content": "Customers may request a refund within 30 days of purchase. Refunds are processed to the original payment method within 5 business days after approval."},
    {"id": "kb2", "title": "Billing issues", "content": "If double-charged, provide transaction ID and order number. We will investigate and process a refund if validated."},
    {"id": "kb3", "title": "Password reset", "content": "To reset your password go to /reset and follow the instructions. If you do not receive an email, check the spam folder or request a manual reset."},
    {"id": "kb4", "title": "Shipping delays", "content": "Shipments can be delayed due to carrier or weather. Typical delays are 3-7 days; provide order number to investigate."},
    {"id": "kb5", "title": "Order cancellation", "content": "Orders can be cancelled within 2 hours of placement. After that, cancellation may not be possible if shipping has started."},
    {"id": "kb6", "title": "Return instructions", "content": "To return an item, pack it securely and use the return label in your account. Returns must be postmarked within 30 days."},
    {"id": "kb7", "title": "Promo codes", "content": "Promo codes apply to eligible items only and cannot be combined. Check terms and expiry."},
    {"id": "kb8", "title": "Warranty", "content": "Products include a 1-year limited warranty covering manufacturing defects."},
    {"id": "kb9", "title": "Account verification", "content": "For security we may ask for order ID and the last 4 digits of the payment method."},
    {"id": "kb10", "title": "International shipping", "content": "International shipping may incur customs duties and longer transit times."},
    {"id": "kb11","title": "EMI Options",
    "content": (
        "Customers can choose EMI options on eligible products. "
        "EMI availability depends on bank, product, and purchase amount. "
        "During checkout, the EMI options will be displayed with interest rates and tenure.")},
    {"id": "kb12","title": "Technical troubleshooting","content": "If you experience errors, clear cache, restart the app, or contact support with error code details."},
    {"id": "kb13","title": "Subscription plans","content": "We offer monthly and yearly subscription plans. Upgrades can be done anytime via your account settings."},
    {"id": "kb14", "title": "Subscription plans","content": "We offer monthly and yearly subscription plans. Upgrades can be done anytime via your account settings."}
]
print("Loaded KB entries:", len(KB))


Loaded KB entries: 14


In [22]:
# Cell 4: classifier training (demo synthetic data)
LABELED = [
    ("I want a refund for my purchase", "refund"),
    ("I was charged twice on my credit card", "billing"),
    ("How do I reset my password", "technical"),
    ("My package has not arrived", "shipping"),
    ("When will my order be delivered", "shipping"),
    ("I need to return the item", "refund"),
    ("There is an unexpected charge on my invoice", "billing"),
    ("I cannot login to my account", "technical"),
]

texts = [t for t, _ in LABELED]
labels = [l for _, l in LABELED]

vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=2000)
X = vectorizer.fit_transform(texts)
clf = LogisticRegression(max_iter=1000)
clf.fit(X, labels)

def predict_label(text: str) -> str:
    x = vectorizer.transform([text])
    return clf.predict(x)[0]

print("Classifier trained.")


Classifier trained.


# EmbeddingSearch

In [23]:
# EmbeddingSearch (tries supported models and falls back to TF-IDF)
class EmbeddingSearch:
    def __init__(self, kb: List[Dict], try_models: Optional[List[str]] = None):
        self.kb = kb
        self.texts = [d["content"] for d in kb]
        self.ids = [d["id"] for d in kb]
        self.kb_embeddings = None

        # TF-IDF prepared always as fallback
        self.tfidf = TfidfVectorizer().fit(self.texts + [d["title"] for d in kb])
        self.tfidf_matrix = self.tfidf.transform(self.texts)

        if try_models is None:
            # model names to try (adjust if your account uses different names)
            try_models = ["gemini-embedding-001", "text-embedding-004", "text-embedding-003"]

        if GENAI_AVAILABLE:
            try:
                genai.configure(api_key=GENAI_API_KEY)
            except Exception as e:
                print("Warning: genai.configure failed:", e)

            for model_name in try_models:
                try:
                    print(f"[EmbeddingSearch] Trying model: {model_name}")
                    emb_list = []
                    for txt in self.texts:
                        resp = genai.embed_content(model=model_name, content=txt)
                        # robust extraction
                        emb = None
                        if isinstance(resp, dict) and "embedding" in resp:
                            emb = np.array(resp["embedding"], dtype="float32")
                        elif hasattr(resp, "embedding"):
                            emb = np.array(resp.embedding, dtype="float32")
                        elif isinstance(resp, dict) and "embeddings" in resp:
                            emb = np.array(resp["embeddings"][0], dtype="float32")
                        elif hasattr(resp, "embeddings"):
                            emb_obj = resp.embeddings[0]
                            if hasattr(emb_obj, "values"):
                                emb = np.array(list(emb_obj.values), dtype="float32")
                            elif hasattr(emb_obj, "embedding"):
                                emb = np.array(emb_obj.embedding, dtype="float32")
                        if emb is None:
                            emb = np.random.rand(768).astype("float32")
                        emb_list.append(emb)
                    self.kb_embeddings = np.vstack(emb_list)
                    self.embedding_model_used = model_name
                    print(f"[EmbeddingSearch] Success with {model_name}, shape = {self.kb_embeddings.shape}")
                    break
                except Exception as e:
                    print(f"[EmbeddingSearch] Model {model_name} failed: {e}")
                    self.kb_embeddings = None
                    continue
            if self.kb_embeddings is None:
                print("[EmbeddingSearch] All embedding attempts failed — TF-IDF will be used.")
        else:
            print("[EmbeddingSearch] GENAI not available — using TF-IDF only.")

    def _embed_query(self, query: str) -> Optional[np.ndarray]:
        if not GENAI_AVAILABLE or self.kb_embeddings is None:
            return None
        try:
            resp = genai.embed_content(model=self.embedding_model_used, content=query)
            if isinstance(resp, dict) and "embedding" in resp:
                return np.array(resp["embedding"], dtype="float32")
            elif hasattr(resp, "embedding"):
                return np.array(resp.embedding, dtype="float32")
            elif isinstance(resp, dict) and "embeddings" in resp:
                return np.array(resp["embeddings"][0], dtype="float32")
            elif hasattr(resp, "embeddings"):
                emb_obj = resp.embeddings[0]
                if hasattr(emb_obj, "values"):
                    return np.array(list(emb_obj.values), dtype="float32")
                elif hasattr(emb_obj, "embedding"):
                    return np.array(emb_obj.embedding, dtype="float32")
            return None
        except Exception as e:
            print("Query embedding failed:", e)
            return None

    def search(self, query: str, top_k: int = 3) -> List[Dict]:
        # Embedding path
        if self.kb_embeddings is not None:
            q_emb = self._embed_query(query)
            if q_emb is not None:
                sims = cosine_similarity([q_emb], self.kb_embeddings).squeeze()
                idx = sims.argsort()[-top_k:][::-1]
                return [self.kb[i] for i in idx]
        # TF-IDF fallback
        qv = self.tfidf.transform([query])
        sims = (self.tfidf_matrix @ qv.T).toarray().squeeze()
        idx = sims.argsort()[-top_k:][::-1]
        return [self.kb[i] for i in idx]

# initialize
embedding_search = EmbeddingSearch(KB)


[EmbeddingSearch] Trying model: gemini-embedding-001
[EmbeddingSearch] Success with gemini-embedding-001, shape = (14, 3072)


In [24]:
class GeminiLLM:
    def __init__(self, model_name="gemini-2.5-flash-lite"):
        # use the already-configured genai client
        self.model = genai.GenerativeModel(model_name)

    def generate(self, prompt, max_output_tokens=256):
        try:
            response = self.model.generate_content(
                prompt,
                generation_config={"max_output_tokens": max_output_tokens}
            )
            return response.text
        except Exception as e:
            raise RuntimeError(f"Gemini generation failed: {e}")


# context compaction helper Function

In [25]:
#  # context compaction helper
def compact_context(customer_memory: Dict, session_history: str, kb_texts: List[str], max_len_chars: int = 3000) -> str:
    parts = []
    if session_history:
        parts.append("Session History:\n" + session_history[-1500:])
    if customer_memory:
        mem_items = [f"{k}: {v}" for k, v in customer_memory.items() if k != "last_response"]
        if mem_items:
            parts.append("Customer Memory:\n" + "\n".join(mem_items))
    if kb_texts:
        parts.append("KB Summary:\n" + "\n".join(kb_texts[:3]))
    ctx = "\n\n".join(parts)
    if len(ctx) > max_len_chars:
        ctx = ctx[-max_len_chars:]
    return ctx


#  ResponseAgent, QAAgent, Memory

In [26]:
# ResponseAgent, QAAgent, Memory
class ResponseAgent:
    def __init__(self, llm_obj: GeminiLLM, embedding_search_obj: EmbeddingSearch):
        self.llm = llm_obj
        self.embedding_search = embedding_search_obj

    def draft(self, customer: Dict, email_text: str, kb_results: List[Dict], session_context: Optional[Dict] = None) -> Dict:
        cust_summary = ""
        if customer:
            parts = [f"{k}: {v}" for k, v in customer.items() if k != "last_response"]
            cust_summary = "\n".join(parts)
        # also retrieve from embedding search (RAG)
        retrieved = self.embedding_search.search(email_text, top_k=3)
        retrieved_text = "\n".join([f"- {r['title']}: {r['content']}" for r in retrieved]) if retrieved else "None"
        kb_summary = "\n".join([f"- {d['title']}" for d in kb_results]) if kb_results else "None"
        compacted = compact_context(customer, session_context or "", [d["content"] for d in retrieved])

        prompt = f"""
You are an expert, concise and polite customer support assistant.

Customer summary:
{cust_summary}

Customer message:
{email_text}

Top KB (titles):
{kb_summary}

Vector retrieved passages:
{retrieved_text}

Compacted context:
{compacted}

Write a concise, polite, and fully grounded customer support reply. Use only the information from the KB or the customer's message. If you need more info, ask a single clarifying question.
"""
        # generate via Gemini
        reply = self.llm.generate(prompt)
         # Ensure signoff is included
        if "regards" not in reply.lower() and "best" not in reply.lower() and "support team" not in reply.lower():
            reply += "\n\nRegards,\nSupport Team"
        return {"reply": reply, "prompt": prompt, "retrieved": retrieved}

class QAAgent:
    def review(self, draft_text: str) -> Dict:
        issues = []
        if len(draft_text.split()) < 8:
            issues.append("reply too short")
        if not any(s in draft_text.lower() for s in ["regards", "Regards","best", "support team"]):
            issues.append("missing signoff")
        return {"issues": issues, "approved": len(issues) == 0}

class CustomerMemory:
    def __init__(self, path: str = "./customer_memory.json"):
        self.path = Path(path)
        if self.path.exists():
            try:
                self.data = json.loads(self.path.read_text())
            except Exception:
                self.data = {}
        else:
            self.data = {}

    def get(self, customer_id: str) -> Dict:
        return self.data.get(customer_id, {})

    def update(self, customer_id: str, info: Dict):
        self.data.setdefault(customer_id, {}).update(info)
        self.path.write_text(json.dumps(self.data, indent=2))

class SessionMemory:
    def __init__(self):
        self.store = {}

    def add(self, session_id: str, message: str):
        self.store.setdefault(session_id, []).append(message)

    def get(self, session_id: str, last_n: int = 5) -> str:
        return "\n".join(self.store.get(session_id, [])[-last_n:])

print("Agents and memory ready.")


Agents and memory ready.


# Session Memory

In [27]:
class SessionStore:
    def __init__(self):
        self.sessions = {}

    def get(self, session_id):
        return self.sessions.get(session_id, {"history": []})

    def save(self, session_id, state):
        self.sessions[session_id] = state


session_store = SessionStore()


# Custom Tool (ToneCheckerTool)

In [28]:
class ToneCheckerTool:
    def analyze(self, text):
        text = text.lower()
        flags = []

        if "angry" in text or "frustrated" in text:
            flags.append("customer emotional tone detected")

        if "urgent" in text or "immediately" in text:
            flags.append("urgent context detected")

        return {"tone_flags": flags}

tone_tool = ToneCheckerTool()


# Observability (Logging + Trace IDs)

In [29]:
import uuid, time

def log_event(event_type, data):
    trace_id = str(uuid.uuid4())[:8]
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")

    

## Architecture Diagram

The system architecture of the AI Customer Support Agent is as follows:

        +-----------------+
        |  Customer Email |
        +--------+--------+
                 |
                 v
       +-------------------+
       |  Logging / Tracing |
       +-------------------+
                 |
                 v
       +-------------------+
       | Tone Detection    |
       | (ToneCheckerTool) |
       +-------------------+
                 |
                 v
       +-------------------+
       | Intent Classification |
       |   (TF-IDF / ML)       |
       +-------------------+
                 |
                 v
       +-------------------+
       | Knowledge Retrieval |
       |  (KB + Google)     |
       +-------------------+
                 |
                 v
       +-------------------+
       | Response Drafting |
       |   (Gemini LLM)   |
       +-------------------+
                 |
                 v
       +-------------------+
       | Quality Review    |
       |   (QAAgent)       |
       +-------------------+
                 |
                 v
       +-------------------+
       | Customer Memory   |
       |  Update & Session |
       +-------------------+
                 |
                 v
       +-------------------+
       | Customer Response |
       +-------------------+


# Orchestration process

In [30]:
# orchestrator_process (uses embedding_search.search)
def orchestrator_process(
    email: Dict,
    classifier_predict,
    search_tool: EmbeddingSearch,
    memory: CustomerMemory,
    response_agent: ResponseAgent,
    qa_agent: QAAgent
):
    session_id = email["customer_id"]
    log_event("email_received", email)
    tone_result = tone_tool.analyze(email["text"])
    
    # 1) classify
    category = classifier_predict(email["text"])
    #log_event("tone_detected", {"category": category})

    # 2) retrieve KB via embeddings (best) or TF-IDF fallback
    kb_hits = search_tool.search(email["text"], top_k=3)
    #log_event("kb_search", {"hits": [d["id"] for d in kb_hits]})
    # 3) session/memory
    cust = memory.get(email["customer_id"])

    # 4) draft response using response_agent (which calls Gemini)
    drafted = response_agent.draft(cust, email["text"], kb_hits)

    # 5) quality review
    qa = qa_agent.review(drafted["reply"])

    # 6) update memory
    memory.update(email["customer_id"], {
        "last_category": category,
        "last_response": drafted["reply"][:300],
        "updated_at": time.time()
    })
    # Save session
    state = session_store.get(session_id)
    state["history"].append(email["text"])
    session_store.save(session_id, state)
    return {
        "email_id": email.get("id") or email.get("email_id"),
        "customer_id": email["customer_id"],
        "text": email["text"],
        "predicted_category": category,
        "kb_hits": [d["id"] for d in kb_hits],
        "retrieved": [r["id"] for r in drafted.get("retrieved", [])],
        "reply": drafted["reply"],
        "qa_approved": qa["approved"],
        "qa_issues": qa["issues"],
    }

print("Orchestrator ready.")


Orchestrator ready.


## Agent Workflow

1. **Email Received**
    - Log the email and generate a trace ID
2. **Tone Analysis**
    - Detect if customer is angry or urgent
3. **Intent Classification**
    - Predict category (refund, shipping, technical, product info)
4. **Knowledge Retrieval**
    - Search internal KB
    - If KB fails, perform Google Search
5. **Response Drafting**
    - Draft response using Gemini LLM
6. **Quality Review**
    - QAAgent checks response quality
7. **Update Memory**
    - Save last response, category, and conversation history
8. **Send Reply**
    - Reply returned to customer


# Agent Responses

In [31]:
# demo run pipeline
memory = CustomerMemory(path="./customer_memory.json")
session_memory = SessionMemory()
llm=GeminiLLM()
response_agent = ResponseAgent(llm, embedding_search)
qa_agent = QAAgent()

SAMPLE_EMAILS = [
    {"id": "e1", "customer_id": "c234", "text": "I can't reset my password. The reset link is not working."},
    {"id": "e2", "customer_id": "c345", "text": "My shipment is late. Where is my package?"},
    {"id": "e3", "customer_id": "c456", "text": "How long is your refund policy? I returned the item last week."},
]

results = []
for e in SAMPLE_EMAILS:
    out = orchestrator_process(e, predict_label, embedding_search, memory, response_agent, qa_agent)
    results.append(out)

df = pd.DataFrame(results)
display(df[["email_id", "customer_id", "predicted_category", "kb_hits", "retrieved", "qa_approved"]])

print("\n--- Example reply (email e1) ---\n")
print(results[2]["reply"])


,email_id,customer_id,predicted_category,kb_hits,retrieved,qa_approved
0,e1,c234,technical,"[kb3, kb12, kb9]","[kb3, kb12, kb9]",True
1,e2,c345,shipping,"[kb4, kb10, kb5]","[kb4, kb10, kb5]",True
2,e3,c456,refund,"[kb1, kb6, kb4]","[kb1, kb6, kb4]",True



--- Example reply (email e1) ---

Our refund policy allows for refunds within 30 days of purchase. Once approved, refunds are typically processed to your original payment method within 5 business days. Since you returned the item last week, your refund should be processed shortly.

Regards,
Support Team


# Testing the agent on various user inputs to understand how the agents will response on different situations and give effective feedback

In [32]:
TEST_EMAILS = [ {
  "id": "1",
  "customer_id": "cx401",
  "text": "I ordered something last week—I think it was the earbuds, but I'm not fully sure because I had multiple items in my cart—and now I can't tell whether the actual product shipped or if only the accessory shipped first. The tracking shows 'in transit' but the weight seems incorrect. Can you verify everything?"
},
{
  "id": "2",
  "customer_id": "c115",
  "text": "I want to purchase 50 units of your product for an event. Do you offer any bulk discount?"
},
{
  "id": "3",
  "customer_id": "c114",
  "text": "I placed an order yesterday but it is not showing under my 'My Orders' section."
},
{
  "id": "4",
  "customer_id": "cx401",
  "text": "I ordered something last week—I think it was the earbuds, but I'm not fully sure because I had multiple items in my cart—and now I can't tell whether the actual product shipped or if only the accessory shipped first. The tracking shows 'in transit' but the weight seems incorrect. Can you verify everything?"
},
{
  "id": "5",
  "customer_id": "cx402",
  "text": "My app keeps crashing whenever I try to upload photos. I already tried reinstalling, clearing cache, and rebooting my phone but it still crashes."
},
{
  "id": "6",
  "customer_id": "cx404",
  "text": "Hello! I accidentally ordered two sets of the same dinnerware. I only need one. Could you please help me return the extra one? It's still sealed."
}              
 ]

for e in TEST_EMAILS:
    result = orchestrator_process(e, predict_label, embedding_search, memory, response_agent, qa_agent)
    print("Email:", e["id"])
    print("Reply:", result["reply"])
    print("QA Approved:", result["qa_approved"])
    print("------")


Email: 1
Reply: I understand you're concerned about your recent order and want to verify its contents and shipping status. To help me investigate this for you, could you please provide your order number?

Regards,
Support Team
QA Approved: True
------
Email: 2
Reply: We do not offer specific bulk discounts. However, you can check for any active promo codes on eligible items.

Regards,
Support Team
QA Approved: True
------
Email: 3
Reply: I understand your order isn't showing in your 'My Orders' section. Could you please provide your order number so I can investigate this for you?

Regards,
Support Team
QA Approved: True
------
Email: 4
Reply: Hello! I can help you verify your order details. To investigate the shipment and the tracking information, could you please provide your order number?

Regards,
Support Team
QA Approved: True
------
Email: 5
Reply: I understand you're experiencing app crashes when uploading photos, even after trying common troubleshooting steps. To help me investi

# Agent Evaluation

In [33]:
memory = CustomerMemory(path="./customer_memory.json")
session_memory = SessionMemory()
llm=GeminiLLM()
response_agent = ResponseAgent(llm, embedding_search)
qa_agent = QAAgent()
test_cases = [
    {
        "email": {"text": "I am frustrated! I want refund immediately!", "customer_id": "c999"},
        "expected": ["customer emotional tone detected"]
    },
    {
        "email": {"text": "When will my order arrive?", "customer_id": "c222"},
        "expected": []
    }
]

def evaluate_agent():
    results = []
    for i, tc in enumerate(test_cases):
        result = orchestrator_process(
            email=tc["email"],
            classifier_predict=predict_label,
            search_tool=embedding_search,
            memory=memory,
            response_agent=response_agent,
            qa_agent=qa_agent
        )
        results.append(result)

    return results

evaluate_agent()


[{'email_id': None,
  'customer_id': 'c999',
  'text': 'I am frustrated! I want refund immediately!',
  'predicted_category': 'refund',
  'kb_hits': ['kb1', 'kb6', 'kb5'],
  'retrieved': ['kb1', 'kb6', 'kb5'],
  'reply': "I understand you're seeking an immediate refund. To process this, could you please provide your order number?\n\nRegards,\nSupport Team",
  'qa_approved': True,
  'qa_issues': []},
 {'email_id': None,
  'customer_id': 'c222',
  'text': 'When will my order arrive?',
  'predicted_category': 'shipping',
  'kb_hits': ['kb4', 'kb10', 'kb5'],
  'retrieved': ['kb4', 'kb10', 'kb5'],
  'reply': 'I can help you with that. To check the status of your order, could you please provide your order number?\n\nRegards,\nSupport Team',
  'qa_approved': True,
  'qa_issues': []}]